# データセット用画像の作成
あらかじめデータセットを作成するディレクトリの配下に移動しておく

In [15]:
# 必要なライブラリのインポート
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import cv2
import natsort

# ファイル保存先の設定

In [16]:
username = ''                                               # ユーザ名
datasetname = 'sndl_size_discrim'                           # データセット名
train_csv = 'train_label.csv'                               # trainラベルを示すcsvファイル名
test_csv = 'test_label.csv'                                 # testラベルを示すcsvファイル名

# path_to_manual = f'/home/{username}/database/{datasetname}/downloads/manual/' # 現在のディレクトリまでのパス
path_to_manual = f'/Users/yonekawahiroto/Sanada_lab/研究/データセット作成パッケージ/{datasetname}/downloads/manual/' # 現在のディレクトリまでのパス

train_data_path = path_to_manual + 'image_folder/train/'    # 作成したtrainデータの保存先
test_data_path = path_to_manual + 'image_folder/test/'      # 作成したtestデータの保存先
print(train_data_path)
print(test_data_path)

/Users/yonekawahiroto/Sanada_lab/研究/データセット作成パッケージ/sndl_size_discrim/downloads/manual/image_folder/train/
/Users/yonekawahiroto/Sanada_lab/研究/データセット作成パッケージ/sndl_size_discrim/downloads/manual/image_folder/test/


# 画像の変数設定

In [17]:
categories = ['small', 'large']                                                             # ラベルのカテゴリ
N_data = 5                                                                                 # 1カテゴリあたりの画像枚数
N_category = len(categories)                                                                # カテゴリ数

image_shape = (224, 224, 3)                                                                 # DCNNに入力する画像のサイズ

left_cntr = (int(image_shape[1] * 1/4)+5, int(image_shape[0]/2))                            # 左側の中心座標(x,y)
right_cntr = (int(image_shape[1] * 3/4)-5, int(image_shape[0]/2))                           # 右側の中心座標(x,y)
cntr = [left_cntr, right_cntr]                                                              # 中心座標

freq=0.2                                                                                    # 生成する画像のテクスチャの空間周波数
vNum=[3,4,5,6,7,9]                                                                          # 生成する多角形の頂点の数

RefDisk_radius_pix = 20                                                                     # Reference diskの半径のピクセル値
TestDisk_radius_arr = np.array([0.461, 0.569, 0.727, 0.825, 1.169, 1.340, 1.530, 1.741])    # Test diskの半径(Reference diskに対する相対値)
TestDisk_radius_pix = TestDisk_radius_arr * RefDisk_radius_pix                              # Test diskの半径のピクセル値

print('基準刺激の半径 : ', RefDisk_radius_pix)
print('テスト刺激の半径 : ', TestDisk_radius_pix)
print('条件組み合わせ数', N_data*len(TestDisk_radius_arr)*2*len(vNum))



基準刺激の半径 :  20
テスト刺激の半径 :  [ 9.22 11.38 14.54 16.5  23.38 26.8  30.6  34.82]
条件組み合わせ数 480


# 生成した画像を格納するディレクトリを作成

In [18]:
'''
./image_folder/
    train/
        category_A/
                |-- category_A_000.jpg
                |-- category_A_001.jpg
        categoty_B/
                |-- category_B_000.jpg
                |-- category_B_001.jpg
    test/
'''
for ii in range(N_category):
    try:
        os.makedirs(train_data_path+categories[ii])   # trainデータ用ディレクトリ
    except:
        print('ディレクトリの生成に失敗しました．')
        pass
    try:
        os.makedirs(test_data_path+categories[ii])   # testデータ用ディレクトリ
    except:
        print('ディレクトリの生成に失敗しました．')
        pass

ディレクトリの生成に失敗しました．
ディレクトリの生成に失敗しました．
ディレクトリの生成に失敗しました．
ディレクトリの生成に失敗しました．


# 画像を生成するための関数

In [19]:
# N角形の頂点を計算
def calculate_N_polygon_vertices(vNum, center, radius, aa, bb):
    vertices = []
    for i in range(vNum):
        angle = 2 * np.pi * i / vNum
        x = center[0] + radius * np.cos(angle)/aa
        y = center[1] + radius * np.sin(angle)/bb
        vertices.append([int(x), int(y)])
    return np.array(vertices)

# N角形を作成
def generate_N_polygon(vNum=int, shape=(224, 224, 3), center=list, radius=list, aa=1, bb=1):
    image = np.zeros(shape)

    left_cntr, right_cntr = center  # 刺激の中心座標
    radius1, radius2 = radius       # 刺激の半径

    # N角形の頂点を計算
    left_polygon_vertices = calculate_N_polygon_vertices(vNum, left_cntr, radius1, aa, bb)
    right_polygon_vertices = calculate_N_polygon_vertices(vNum, right_cntr, radius2, aa, bb)
    
    # 頂点のリシェイプ
    left_pts = left_polygon_vertices.reshape((-1, 1, 2))
    right_pts = right_polygon_vertices.reshape((-1, 1, 2))

    # 多角形の頂点を持つ配列をリシェイプして内部を塗りつぶす
    image=cv2.fillPoly(image, [left_pts], color=(1, 1, 1))
    image=cv2.fillPoly(image, [right_pts], color=(1, 1, 1))

    image_outside = (image == 0)
    return image, image_outside

# 正弦波グレーティング
def generate_sinwave(XX, YY, freq, Theta_deg_Sin):
    offset=90
    theta_rad = np.radians(Theta_deg_Sin - offset)
    XX2=np.cos(theta_rad)*XX
    YY2=np.sin(theta_rad)*YY
    ZZ = np.sin(2*np.pi*freq*(XX2+YY2))
    return ZZ

# 正弦波グレーティングを複数掛け合わせたテクスチャの生成
def mask_texture(fact_N, shape, XX, YY, freq, theta_deg):
    texture = np.ones(shape)

    for ii in range(fact_N):
        texture *= generate_sinwave(XX, YY, freq, theta_deg[ii])
    return texture

# trainデータの生成

In [20]:
xx = np.arange(0, image_shape[1])
yy = np.arange(0, image_shape[0])
zz = np.arange(0, image_shape[2])
[XX,YY,ZZ]=np.meshgrid(xx,yy,zz)

label = None    # ラベルの初期化
ifig=0          # 画像番号を初期化

for ii in range(len(TestDisk_radius_pix)):
    for jj in range(len(vNum)):
        # 各条件の組み合わせをN_dataずつ作成
        for __ in range(N_data):
            fact_N = np.random.randint(1, 5)                                                                                        # 正弦波を掛け合わせる回数(テクスチャ設定用)
            theta_deg = np.random.randint(0, 360, fact_N)                                                                           # 正弦波の角度
            
            # 左がrefenrence, 右がtestの条件
            image, image_outside = generate_N_polygon(vNum[jj], image_shape, cntr, [RefDisk_radius_pix, TestDisk_radius_pix[ii]])   # test刺激が右側にある条件
            image *= mask_texture(fact_N, image_shape, XX, YY, freq, theta_deg)                                                     # テクスチャを適用
            image[image_outside] = 0.5                                                                                              # 背景をグレーにする
            
            # 左より右が大きいか?
            if RefDisk_radius_pix <= TestDisk_radius_pix[ii]:
                label='large'
            else:
                label='small'

            cv2.imwrite(train_data_path + f'{label}/{label}_{ifig:03}.jpg', image*255)                                              # 画像の保存
            ifig+=1                                                                                                                 # 画像番号をインクリメント

            # 左がtest, 右がreferenceの条件
            image, image_outside = generate_N_polygon(vNum[jj], image_shape, cntr, [TestDisk_radius_pix[ii], RefDisk_radius_pix])   # test刺激が右側にある条件
            image *= mask_texture(fact_N, image_shape, XX, YY, freq, theta_deg)                                                     # テクスチャを適用
            image[image_outside] = 0.5                                                                                              # 背景をグレーにする
            
            # 左より右が大きいか?
            if TestDisk_radius_pix[ii] <= RefDisk_radius_pix:
                label='large'
            else:
                label='small'

            cv2.imwrite(train_data_path + f'{label}/{label}_{ifig:03}.jpg', image*255)                                              # 画像の保存
            ifig+=1                                                                                                                 # 画像番号をインクリメント


# trainデータにラベルを付与しcsvを作成

In [21]:
train_image_list = glob.glob(train_data_path+'*/*')                                             # trainデータのパスをリストで取得
train_image_list = natsort.natsorted(train_image_list)                                          # 画像を連番になるようにソート

train_label_list = [os.path.split(os.path.split(fname)[0])[1] for fname in train_image_list]    # ラベルを画像数分用意

file_label_list = zip(train_image_list, train_label_list)                                       # trainデータにラベルを対応付け

df = pd.DataFrame(file_label_list, columns=['file_path', 'label'])                              # データフレーム化
df.to_csv(path_to_manual + train_csv)                                                           # csvファイルに保存
df.head()

,file_path,label
0,/Users/yonekawahiroto/Sanada_lab/研究/データセット作成ハ...,large
1,/Users/yonekawahiroto/Sanada_lab/研究/データセット作成ハ...,large
2,/Users/yonekawahiroto/Sanada_lab/研究/データセット作成ハ...,large
3,/Users/yonekawahiroto/Sanada_lab/研究/データセット作成ハ...,large
4,/Users/yonekawahiroto/Sanada_lab/研究/データセット作成ハ...,large


# testデータの生成

In [22]:
vNum = [360]        # テストだけ円形刺激を使用

xx = np.arange(0, image_shape[1])
yy = np.arange(0, image_shape[0])
zz = np.arange(0, image_shape[2])
[XX,YY,ZZ]=np.meshgrid(xx,yy,zz)

label = None    # ラベルの初期化
ifig=0          # 画像番号を初期化

for ii in range(len(TestDisk_radius_pix)):
    for jj in range(len(vNum)):
        # 各条件の組み合わせをN_dataずつ作成
        for __ in range(N_data):
            fact_N = np.random.randint(1, 5)                                                                                        # 正弦波を掛け合わせる回数(テクスチャ設定用)
            theta_deg = np.random.randint(0, 360, fact_N)                                                                           # 正弦波の角度
            
            # 左がrefenrence, 右がtestの条件
            image, image_outside = generate_N_polygon(vNum[jj], image_shape, cntr, [RefDisk_radius_pix, TestDisk_radius_pix[ii]])   # test刺激が右側にある条件
            image *= mask_texture(fact_N, image_shape, XX, YY, freq, theta_deg)                                                     # テクスチャを適用
            image[image_outside] = 0.5                                                                                              # 背景をグレーにする
            
            # 左より右が大きいか?
            if RefDisk_radius_pix <= TestDisk_radius_pix[ii]:
                label='large'
            else:
                label='small'

            cv2.imwrite(test_data_path + f'{label}/{label}_{ifig:03}.jpg', image*255)                                              # 画像の保存
            ifig+=1                                                                                                                 # 画像番号をインクリメント

            # 左がtest, 右がreferenceの条件
            image, image_outside = generate_N_polygon(vNum[jj], image_shape, cntr, [TestDisk_radius_pix[ii], RefDisk_radius_pix])   # test刺激が右側にある条件
            image *= mask_texture(fact_N, image_shape, XX, YY, freq, theta_deg)                                                     # テクスチャを適用
            image[image_outside] = 0.5                                                                                              # 背景をグレーにする
            
            # 左より右が大きいか?
            if TestDisk_radius_pix[ii] <= RefDisk_radius_pix:
                label='large'
            else:
                label='small'

            cv2.imwrite(test_data_path + f'{label}/{label}_{ifig:03}.jpg', image*255)                                              # 画像の保存
            ifig+=1                                                                                                                 # 画像番号をインクリメント


# testデータにラベルを付与しcsvを作成

In [23]:
test_image_list = glob.glob(train_data_path+'*/*')                                              # testデータのパスをリストで取得
test_image_list = natsort.natsorted(test_image_list)                                            # 画像を連番になるようにソート

test_label_list = [os.path.split(os.path.split(fname)[0])[1] for fname in test_image_list]      # ラベルを画像数分用意

file_label_list = zip(test_image_list, test_label_list)                                         # testデータにラベルを対応付け

df = pd.DataFrame(file_label_list, columns=['file_path', 'label'])            # データフレーム化
df.to_csv(path_to_manual + test_csv)                                                            # csvファイルに保存
df.head()

,file_path,label
0,/Users/yonekawahiroto/Sanada_lab/研究/データセット作成ハ...,large
1,/Users/yonekawahiroto/Sanada_lab/研究/データセット作成ハ...,large
2,/Users/yonekawahiroto/Sanada_lab/研究/データセット作成ハ...,large
3,/Users/yonekawahiroto/Sanada_lab/研究/データセット作成ハ...,large
4,/Users/yonekawahiroto/Sanada_lab/研究/データセット作成ハ...,large
